In [1]:
def pre_processing(df):
    #remove columns
    df = df.drop(['codigo_viagem','tipo_viagem','data_viagem_programada','hora_viagem_programada','pdop','numero_imei','in_transbordo','codigo_viagem_transbordo'],axis=1)
    
    df['data_inicio_viagem'] = pd.to_datetime(df['data_inicio_viagem'])
    df['data_fim_viagem'] = pd.to_datetime(df['data_fim_viagem'])
    
    df = df.sort_values('data_inicio_viagem')
    
    mask = (df['latitude'] >= -15.8999) & (df['latitude'] <= -15.8711) & (df['longitude'] >= -52.3206) & (df['longitude'] <= -52.2544)
    
    bg = df[mask]
    
    bg = bg.drop_duplicates(subset=['data_inicio_viagem', 'data_fim_viagem'], keep='first')
    
    linhas = pd.read_csv('linhas_secoes.csv', encoding='latin', sep=',')
    
    linhas['prefixo'] = linhas['prefixo'].str.replace('-', '').astype(int)
    
    # Group the linhas DataFrame by prefixo and get the first row of each group
    linhas_df = linhas.groupby('prefixo').first().reset_index()

    # Merge the viagens and linhas DataFrames on the prefixo and nu_linha columns
    viagens_df = pd.merge(bg, linhas_df[['prefixo', 'descricao_linha']], 
                          left_on='nu_linha', right_on='prefixo')

    # Drop the prefixo column
    viagens_df.drop('prefixo', axis=1, inplace=True)

    # Rename the descricao_linha column to desc_linha
    viagens_df.rename(columns={'descricao_linha': 'desc_linha'}, inplace=True)
    
    viagens_sorted = viagens_df.sort_values('data_inicio_viagem')
    
    return viagens_sorted